<div align="center">

# United States International University-Africa  
### School of Science and Technology  

---
## *DSA2020B - INTRODUCTION TO ARTIFICIAL INTELLIGENCE*
## *Constriant_satification_Lab_Scheduler_Project*

---
## Group 0ne:
### Group Members:

#### **Betelhem Kebede** - *670549*  
#### **Snit Kahsay** - *670552*  
#### **Hana Gashaw** - *670555*
#### **Vivian Jerono Kipsang** -*670386*
#### **Merhawit Tesfay** -*670554*
</div>


### *Team Members & Responsibilities*

 **Requirements & Constraints Identification** – *Betelhem & Vivian*  
  - Defined hard constraints and scheduling rules.

 **Data Preparation** – *Hana & Merhawit*  
  - Collected and structured input data (courses, labs, lecturers).  
  - Created domain variables and helper mappings.

  **Algorithm Implementation** – *Snit & Betelhem*  
  - Built the backtracking scheduling algorithm.  
  - Handled constraint enforcement and session pairing logic.

 **GUI Development** – *Hana , Vivian &  Snit*  
  - Designed and implemented the user interface.  
  - Enabled input loading and schedule display.

 **Testing & Documentation** – *Snit & Merhawit*  
  - Performed validation and edge case testing.  
  - Authored user and technical documentation.




This project presents a *Constraint Satisfaction Lab Scheduler* that was developed to automatically generate valid weekly timetables for *Data Science and Analytics courses* at *USIU-Africa*  courses categorized as STA, DSA, MTH, APT, and IST.

Initially, the system was created to assign over *40 courses*, some of which have multiple sections, bringing the total number of classes to *65, to 5 laboratory rooms* located in the *Lillian K. Beam Building*.

For further flexibility and usability, the system was later enhanced to accept *custom user input, allowing any number of **data science courses, **labs, and **lecturers* to be defined. This enhancement was introduced to accommodate the dynamic nature of teaching staff, fluctuating course loads across different semesters, and to make the system adaptable for scheduling in *other campus buildings beyond Lillian K. Beam*.

The scheduler was designed with the following core objectives:
- Each course section must be assigned a valid *day, **time slot, **lab, and **lecturer*.
- *No two course sections* should share the same lab or lecturer at the same time.
- *Double-session classes* must be consistently scheduled on either *Monday/Wednesday* or *Tuesday/Thursday*.
- *Full-session classes* must be scheduled on *Friday or Saturday* only.
- *Back-to-back teaching blocks* for lecturers must be minimized to avoid overload.

To achieve these goals, the following input is processed:
- A list of approximately *65 course sections*
- Pools of lecturers grouped by *department code* (e.g., DSA, MTH, APT)
- Available *days* (Monday to Saturday) and *time slots*
- A list of available *laboratory rooms*

From this input, a *domain of possible assignments* is constructed for each course — consisting of all valid combinations of *(day, time slot, lab, lecturer)*.

A set of constraint functions is applied to ensure a feasible schedule:
- room_conflict() — avoids assigning two classes to the same lab at the same time.
- lecturer_conflict() — prevents instructors from being double-booked.
- same_course_consistency() — ensures consistent day-pairing for each course.
- back_to_back_ok() — avoids assigning too many consecutive sessions to lecturers.
- paired_day_of() — maps sessions to valid day pairs (e.g., Monday → Wednesday).

A *backtracking algorithm* is used as the core solver. It recursively assigns courses to valid slots while checking for constraint violations. If a violation is detected, the algorithm *backtracks* and tries alternate assignments, continuing until a complete and valid schedule is found — or it confirms that no valid arrangement is possible.

### *Why Backtracking?*

Backtracking was chosen because:

* It can efficiently search large solution spaces.SPACE AND TIME COMPLEITY
* It handles complex, interdependent constraints.
* It guarantees either a valid full solution or proof that no such schedule exists.




The resulting timetable includes:
- Assigned *days*
- *Time slots*
- *Laboratory rooms*
- *Lecturers*

This scheduling system demonstrates how constraint satisfaction techniques, combined with backtracking, can be effectively used to solve complex, real-world timetable generation problems — with scalability and adaptability in mind.



### *Weekly Structure – Class Time Model*

---

#### *Monday to Thursday*

* *Day Type*: Weekday double-session classes
* *Class Type*: One course taught *twice per week*
* *Sessions*: Up to *7 sessions per day(7:00 am-9:00 pm)*
* *Session Duration*: Monday to Thursday(1:40 minutes/sesission)  & Friday and Saturday(3:20 minutes/session)
* *Time Frame*: 7:00 AM – 9:00 PM
* *Day Pairs*:

  * Mon/Wed
  * Tue/Thu

*Each class has two sessions/week, e.g., Monday and Wednesday.*

---

#### *Friday – Weekend Day*

* *Class Type*: Full-session (single long meeting)
* *Sessions*:

  * 8:00 AM – 11:20 AM
  * 1:20 PM – 4:40 PM
  * 5:40 PM – 9:00 PM
* *Duration per session*: 3 hours 20 minutes

---

#### *Saturday – Weekend Day*

* *Class Type*: Full-session class
* *Sessions*:

  * 9:00 AM – 12:20 PM
  * 1:20 PM – 4:40 PM


----



## *Import necessary modules*

In [1]:
import sys                           # Provides access to system-specific parameters and functions (e.g., command-line arguments)
from collections import defaultdict  # Allows creation of dictionaries with default  values, useful for grouping or counting

## *Variable Description – Lab Scheduler Project*

The scheduler system is built on several core variables, each playing a role in organizing and assigning lab sessions effectively.

---

### **1. `course_sections`**
A list containing all the course sections that need lab scheduling.  
Each item is a string formatted as `"COURSECODE_Section"`, e.g., `"APT1050_A"`, representing a unique lab section.

- Covers departments: `APT`, `DSA`, `MTH`, `STA`, `IST`.
- Useful for mapping each section to a time slot, lab, and instructor.

---

### **2. `lecturers`**
A dictionary where:
- Keys = department codes (e.g., `'APT'`, `'DSA'`).
- Values = lists of lecturers assigned to each department (e.g., `["APT_Lec1", "APT_Lec2"]`).

This helps in ensuring department-specific instructor assignments and avoiding conflicts.

---

### **3. `slots_by_day`**
A dictionary representing all possible time slots for each day of the week.

- Keys = Days (`"Mon"`, `"Tue"`, ..., `"Sat"`).
- Values = List of available time slots (strings), formatted like `"8:00-9:40"` or `"10:00-11:40"`.

Special formats:
- Mon–Thu: Two time blocks per day.
- Fri–Sat: One long block per day (e.g., `"8:00-10:40"` for 3:20-hour sessions).

---

### **4. `labs`**
A list of integers representing lab room numbers:  
`[1, 2, 3, 4, 5]`

These represent physical spaces (Lab 1 to Lab 5) where sessions can be held.  
The scheduler ensures no double-booking of labs across departments or time slots.

---


In [2]:
course_sections = [
    "APT1050_A", "APT1050_B", "APT1050_C", "APT2060_A", "APT2060_B",
    "APT3040_A", "APT3040_B", "DSA1060_A", "DSA1060_B",
    "DSA1080_A", "DSA1080_B", "DSA2020_A", "DSA2040_A",
    "DSA3020_A", "DSA3020_B", "DSA3030_A", "DSA3050_A",
    "DSA3900_A", "DSA4020_A", "DSA4030_A", "DSA4050_A",
    "DSA4900_A", "DSA4910_A", "DSA4b00_A", "IST1020_A", "IST1020_B",
    "IST1020_C", "MTH1040_A", "MTH1040_B", "MTH1050_A", "MTH1050_B",
    "MTH1050_C", "MTH1060_A", "MTH1060_B", "MTH1109_A", "MTH1109_B",
    "MTH1110_A", "MTH1110_B", "MTH1110_C", "MTH2020_A", "MTH2020_B",
    "MTH2030_A", "MTH2030_B", "MTH2215_A", "MTH2215_B", "MTH2215_C",
    "MTH3010_A", "MTH3010_B", "STA1020_A", "STA1040_A", "STA2010_A",
    "STA2010_B", "STA2030_A", "STA2050_A", "STA2050_B", "STA2060_A",
    "STA3010_A", "STA3020_A", "STA3040_A", "STA3040_B", "STA3050_A",
    "STA4010_A", "STA4020_A", "STA4030_A", "STA4030_B"
]

lecturers = {
    'STA': ['STA_Lec1', 'STA_Lec2', 'STA_Lec3', 'STA_Lec4', 'STA_Lec5', 'STA_Lec6', 'STA_Lec7', 'STA_Lec8'],
    'DSA': ['DSA_Lec1', 'DSA_Lec2', 'DSA_Lec3', 'DSA_Lec4', 'DSA_Lec5', 'DSA_Lec6', 'DSA_Lec7', 'DSA_Lec8', 'DSA_Lec9', 'DSA_Lec10'],
    'MTH': ['MTH_Lec1', 'MTH_Lec2', 'MTH_Lec3', 'MTH_Lec4', 'MTH_Lec5', 'MTH_Lec6', 'MTH_Lec7', 'MTH_Lec8', 'MTH_Lec9', 'MTH_Lec10', 'MTH_Lec11', 'MTH_Lec12', 'MTH_Lec13', 'MTH_Lec14', 'MTH_Lec15'],
    'APT': ['APT_Lec1', 'APT_Lec2', 'APT_Lec3', 'APT_Lec4', 'APT_Lec5', 'APT_Lec6', 'APT_Lec7', 'APT_Lec8', 'APT_Lec9'],
    'IST': ['IST_Lec1', 'IST_Lec2', 'IST_Lec3', 'IST_Lec4', 'IST_Lec5', 'IST_Lec6','IST_Lec7']
}


slots_by_day = {
    "Mon": ['7:00–8:40', '9:00–10:40', '11:00–12:40', '1:20–3:00', '3:30–5:10', '5:30–7:10', '7:30–9:00'],
    "Tue": ['7:00–8:40', '9:00–10:40', '11:00–12:40', '1:20–3:00', '3:30–5:10', '5:30–7:10', '7:30–9:00'],
    "Wed": ['7:00–8:40', '9:00–10:40', '11:00–12:40', '1:20–3:00', '3:30–5:10', '5:30–7:10', '7:30–9:00'],
    "Thu": ['7:00–8:40', '9:00–10:40', '11:00–12:40', '1:20–3:00', '3:30–5:10', '5:30–7:10', '7:30–9:00'],
    "Fri": ['8:00–11:20', '1:20–4:40', '5:40–9:00'],
    "Sat": ['9:00–12:20', '1:20–4:40']
}

labs = [1, 2, 3, 4, 5]


##  *Helper Function Descriptions (for Scheduler Project)*

---

### 1. `get_lecturers(course)`

> **Definition**:
> This function extracts the prefix (e.g., `"MTH"`, `"APT"`) from the course code and retrieves a list of eligible lecturers assigned to that course. If no lecturer is found for the given prefix, it returns a default value of `['Unknown']`.

> **Purpose**:
> To ensure that only valid lecturers are assigned when building scheduling options for each course, thereby preserving academic alignment and accuracy.

---

### 2. `day_pattern(day)`

> **Definition**:
> This function maps each day to a broader scheduling pattern group:

* **Monday / Wednesday** → `"MW"`
* **Tuesday / Thursday** → `"TTh"`
* **Friday / Saturday** → `"FriSat"`
* Other days are classified as `"Other"` (fallback).

> **Purpose**:
> Used for enforcing **pattern-based constraints**, ensuring that courses scheduled on multiple days follow consistent weekly scheduling structures.

---

### 3. `paired_day_of(day)`

> **Definition**:
> This function returns the *paired day* for common weekday scheduling patterns. For example:

* `"Mon"` ⇄ `"Wed"`
* `"Tue"` ⇄ `"Thu"`

Fridays and Saturdays are not paired with any other day.

> **Purpose**:
> To support **bi-weekly scheduling logic** (e.g., MW or TTh classes) by enabling symmetric pairing between weekdays.


In [3]:
# --- Helpers ---
def get_lecturers(course):
    prefix = course[:3]
    # return all lecturers for that prefix; if none, return a fallback single 'Unknown'
    return lecturers.get(prefix, ['Unknown'])

def day_pattern(day):
    if day == "Mon" or day == "Wed":
        return "MW"
    elif day == "Tue" or day == "Thu":
        return "TTh"
    elif day == "Fri":
        return "Fri"
    elif day == "Sat":
        return "Sat"
    else:
        return "Other"

def paired_day_of(day):
    """Symmetric pairing: Mon<->Wed, Tue<->Thu. Fri/Sat have no pair."""
    if day == "Mon": return "Wed"
    if day == "Wed": return "Mon"
    if day == "Tue": return "Thu"
    if day == "Thu": return "Tue"
    return None

## *The domain for each course section is generated using*


*Each variable (course section) has a domain consisting of all possible 4-tuples:*
- *(day, time_slot, lab_room, lecturer)*

 Where:

- *day ∈ ["Mon", "Tue", ..., "Sat"]*

- *time_slot ∈ slots_by_day[day] (different time blocks per day)*

- *lab_room ∈ [1, 2, 3, 4, 5]*

- *lecturer ∈ lecturers[department_prefix_of(course)]*

Thus, domain size = #days × #slots × #labs × #lecturers_for_course.

In [4]:
# Build domain for each course using ALL lecturers
def build_domain_for_course(course):
    lec_list = get_lecturers(course)  # take all lecturers
    domain = []
    for day in slots_by_day:
        for time in slots_by_day[day]:
            for lab in labs:
                for lec in lec_list:
                    domain.append((day, time, lab, lec))
    return domain

### *CONSTRAINTS*

*The constraints are enforced inside the recursive backtracking function, particularly via these helper checks:*

### *I.Room Conflict Constraint*
* *No two sessions can be assigned to the *same lab* at the *same time* on the same day.*
### *II.Lecturer Conflict Constraint*
* *Prevents a lecturer from being double-booked (teaching more than one  class at a time).*

### *III.Back_to_back_ok*
* *Prevents a lecturer from having more than 2 `consecutive lab sessions` on the same day.*
### *IV.Same_course_consistency*
* *If a course is assigned on Mon or Tue, its paired session must be assignable on Wed/Thu.*
### *V.paired_day_of(day)*
* *Double classes take place on Mon/Wed or Tue/Thur*



In [5]:
# --- Constraint checks used by the backtracker ---
def room_conflict(candidate, room_schedule):
    return (candidate[0], candidate[1], candidate[2]) in room_schedule

def lecturer_conflict(candidate, lec_time_set):
    return (candidate[0], candidate[1], candidate[3]) in lec_time_set

def back_to_back_ok(candidate, lec_schedule, max_back_to_back=2):
    day, time, _, lec = candidate
    if day not in lec_schedule.get(lec, {}):
        return True
    slot_index = {t:i for i,t in enumerate(slots_by_day[day])}
    if time not in slot_index:  # safety
        return False
    this_idx = slot_index[time]
    existing = sorted(lec_schedule[lec][day] + [this_idx])
    count = 1
    for i in range(1, len(existing)):
        if existing[i] == existing[i-1] + 1:
            count += 1
            if count > max_back_to_back:
                return False

        else:
            count = 1
    return True


def same_course_consistency(candidate, course, assignments):
    """Ensure a course doesn't mix MW/TTh/FriSat and doesn't place two independent sessions on paired days."""
    cand_day, cand_time, cand_lab, cand_lec = candidate
    cand_pat = day_pattern(cand_day)
    # Check existing assignments for the same course (including synthetic duplicates)
    for key, val in assignments.items():
        # key is course name or course+'::Dup'
        # We only compare entries for the same course
        if not key.startswith(course):
            continue
        od, ot, ol, olect = val
        # cannot occupy same day/time/lab (prevents independent duplicate landing on pair day)
        if od == cand_day and ot == cand_time and ol == cand_lab:
            return False
        # MW vs TTh consistency
        p_other = day_pattern(od)
        if (p_other == "MW" and cand_pat == "TTh") or (p_other == "TTh" and cand_pat == "MW"):
            return False
        # Fri/Sat mixing
        if (p_other == "FriSat" and cand_pat in ("MW", "TTh")) or (cand_pat == "FriSat" and p_other in ("MW", "TTh")):
            return False
    return True


## *Backtracking Assignment Algorithm*

The recursive function `backtrack_assign(...)` implements **backtracking search** to solve a **course scheduling** problem with constraints. It attempts to assign each course session to a suitable `time slot`, `room`, and `lecturer` while avoiding conflicts.

---

### Step-by-Step Logic

1. **Base Case:**
   If all sessions in the `order` list are  scheduled the algorithm stops and returns success.

2. **Select a Course Session:**
   It selects the next session to work on from the pre-determined order list.
3. **Try all candidate slots:**
   For each candidate `(Day, Time, Room (Lab), Lecturer)And for each option, it checks whether assigning this session would violate any rule.

   * **Room Conflict:** Ensure the room isn't double-booked.
   * **Lecturer Conflict:** Ensure a lecturer isn’t double-booked.
   * **Back-to-Back Rule:** Verify that assigned times are appropriately spaced.
   * **Same Course Consistency:** Ensures that sessions from the same course don’t overlap or conflict in time.

4. **Check Paired Day (if exists):**
  If a session requires a paired session on another day (e.g., Monday & Wednesday):

  It applies the same validation checks to both days.

   Ensures the same lecturer, time slot, and room (if required).

5. **Make the Assignment:**
     If all checks pass:
               - The chosen assignment is saved.
               - The room and lecturer schedules are updated to reflect the new booking.


7. **Recursive Call:**
   Moves on to the next session and repeats the process.
8. **Backtrack (Undo):**
   If no valid assignment is found for the session, undo all changes:

   * Reverts room and lecturer schedules.
   * Update all sets (restore previous state)

9. **Return Failure:**
   If no option works, it concludes that no solution exists for the current path and backs up further.

In [6]:
# --- Backtracking solver ---
def backtrack_assign(index, order, domains, assignments,
                     room_schedule, lec_time_set, lec_schedule, slot_index_map):
    """Recursive backtracking: assign order[index], return True if solution."""
    if index >= len(order):
        return True  # all assigned

    sv = order[index]  # session variable (here we use course names as session vars)
    course = sv
    for cand in domains[sv]:
        # check candidate validity for main day
        if room_conflict(cand, room_schedule): continue
        if lecturer_conflict(cand, lec_time_set): continue
        if not back_to_back_ok(cand, lec_schedule): continue
        if not same_course_consistency(cand, course, assignments): continue

        # check paired day if applicable
        pday = paired_day_of(cand[0])
        pair = None
        if pday:
            pair = (pday, cand[1], cand[2], cand[3])
            # ensure paired-day availability
            if room_conflict(pair, room_schedule): continue
            if lecturer_conflict(pair, lec_time_set): continue
            if not back_to_back_ok(pair, lec_schedule): continue
            # also check same_course_consistency against pair (prevents independent sessions colliding)
            if not same_course_consistency(pair, course, assignments): continue

        # --- commit candidate (and its pair if any) ---
        assignments[course] = cand
        room_schedule.add((cand[0], cand[1], cand[2]))
        lec_time_set.add((cand[0], cand[1], cand[3]))
        lec_schedule[cand[3]][cand[0]].append(slot_index_map[cand[0]][cand[1]])

        dup_key = None
        if pair:
            dup_key = course + "::Dup"
            assignments[dup_key] = pair
            room_schedule.add((pair[0], pair[1], pair[2]))
            lec_time_set.add((pair[0], pair[1], pair[3]))
            lec_schedule[pair[3]][pair[0]].append(slot_index_map[pair[0]][pair[1]])

        # recurse
        if backtrack_assign(index+1, order, domains, assignments, room_schedule, lec_time_set, lec_schedule, slot_index_map):
            return True

        # --- undo commit ---
        # remove main
        assignments.pop(course, None)
        room_schedule.remove((cand[0], cand[1], cand[2]))
        lec_time_set.remove((cand[0], cand[1], cand[3]))
        lec_schedule[cand[3]][cand[0]].pop()  # remove last appended
        # remove pair if added
        if dup_key:
            assignments.pop(dup_key, None)
            room_schedule.remove((pair[0], pair[1], pair[2]))
            lec_time_set.remove((pair[0], pair[1], pair[3]))
            lec_schedule[pair[3]][pair[0]].pop()

    # no candidate worked for this session var
    return False



## `solve_backtracking()` 

This function schedules all course sections using backtracking while enforcing key constraints like room and lecturer availability.

### What it Does:

1. **Builds domains**
   Creates all valid combinations of time slots, labs, and lecturers for each course section.

   These combinations define the possible ways each course can be scheduled.

3. **Orders the courses**
   Sorts them by difficulty (`More constrianed variable`) to reduce conflicts early.

4. **Tracks availability**
   Uses sets and maps to monitor:

   * Room usage (no double bookings)
   * Lecturer schedules (no overlaps)
   * Daily time slot positions

5. **Runs backtracking**
   Assigns courses one by one. If a conflict is found, it undoes and tries another option.

6. **Returns results**
   If successful, groups assignments by course. If no valid schedule can be created due to too many constraints or conflicts, the function returns None.

---

In [7]:
def solve_backtracking():
    # build domains using ALL lecturers
    domains = {}
    for c in course_sections:
        domains[c] = build_domain_for_course(c)

    # order vars (fail-first heuristic: smallest domain first)
    order = sorted(course_sections, key=lambda x: len(domains[x]))

    # setup schedules and helper maps
    assignments = {}
    room_schedule = set()
    lec_time_set = set()
    lec_schedule = defaultdict(lambda: defaultdict(list))
    slot_index_map = {day: {t:i for i,t in enumerate(slots_by_day[day])} for day in slots_by_day}

    ok = backtrack_assign(0, order, domains, assignments, room_schedule, lec_time_set, lec_schedule, slot_index_map)
    if not ok:
        return None
    # convert to course-level grouped output
    course_assignments = defaultdict(list)
    for k, v in assignments.items():
        if k.endswith("::Dup"):
            base = k.split("::")[0]
            course_assignments[base].append((k, v))
        else:
            course_assignments[k].append((k, v))
    return course_assignments

## `__main__` – Summary

This block runs the backtracking solver and prints the schedule in a readable format.

### What it Does:

1. **Starts Solver**
   Runs `solve_backtracking()` to try assigning all courses.

2. **Handles Failure**
   If no valid schedule is found, it suggests relaxing constraints and exits.

3. **Displays Results**
   For each course:

   * If it has two sessions (main & duplicate), it combines their days (like `Mon/Wed`)
   * If only one, prints day, time, lab, and lecturer clearly

4. **Finishes with "Done"**
   Prints a confirmation after completing the schedule.


In [8]:
# --- Run solver and print nicely ---
if __name__ == "__main__":
    print("Running backtracking solver (this may take time)...")
    sol = solve_backtracking()
    if not sol:
        print("No solution found. Consider relaxing constraints or pruning domains.")
        sys.exit(1)
         # print results. Normalise paired days ordering as Mon/Wed or Tue/Thu
    order_days = ["Mon","Tue","Wed","Thu","Fri","Sat"]
    for course in sorted(sol):
        # group main & dup
        main = None
        dup = None
        for k, val in sol[course]:
            if k.endswith("::Dup"):
                dup = val
            else:
                main = val
        if main and dup:
            days = sorted([main[0], dup[0]], key=lambda x: order_days.index(x))
            day_str = f"{days[0]}/{days[1]}"
            time = main[1]  # same for both
            lab = main[2]
            lect = main[3]
            print(f"{course:12s} -> Day={day_str:9s} Time={time:10s} Lab={lab} Lecturer={lect}")
        elif main:
            print(f"{course:12s} -> Day={main[0]:3s} Time={main[1]:10s} Lab={main[2]} Lecturer={main[3]}")
        elif dup:
            print(f"{course:12s} -> Day={dup[0]:3s} Time={dup[1]:10s} Lab={dup[2]} Lecturer={dup[3]}")
    print("\nDone.")

Running backtracking solver (this may take time)...
APT1050_A    -> Day=Mon/Wed   Time=3:30–5:10  Lab=1 Lecturer=APT_Lec1
APT1050_B    -> Day=Mon/Wed   Time=3:30–5:10  Lab=2 Lecturer=APT_Lec2
APT1050_C    -> Day=Mon/Wed   Time=3:30–5:10  Lab=3 Lecturer=APT_Lec3
APT2060_A    -> Day=Mon/Wed   Time=3:30–5:10  Lab=4 Lecturer=APT_Lec4
APT2060_B    -> Day=Mon/Wed   Time=3:30–5:10  Lab=5 Lecturer=APT_Lec5
APT3040_A    -> Day=Mon/Wed   Time=5:30–7:10  Lab=1 Lecturer=APT_Lec1
APT3040_B    -> Day=Mon/Wed   Time=5:30–7:10  Lab=2 Lecturer=APT_Lec2
DSA1060_A    -> Day=Mon/Wed   Time=5:30–7:10  Lab=3 Lecturer=DSA_Lec1
DSA1060_B    -> Day=Mon/Wed   Time=5:30–7:10  Lab=4 Lecturer=DSA_Lec2
DSA1080_A    -> Day=Mon/Wed   Time=5:30–7:10  Lab=5 Lecturer=DSA_Lec3
DSA1080_B    -> Day=Mon/Wed   Time=7:30–9:00  Lab=1 Lecturer=DSA_Lec1
DSA2020_A    -> Day=Mon/Wed   Time=7:30–9:00  Lab=2 Lecturer=DSA_Lec2
DSA2040_A    -> Day=Mon/Wed   Time=7:30–9:00  Lab=3 Lecturer=DSA_Lec3
DSA3020_A    -> Day=Mon/Wed   Time=7:3

### Constraint-Sensitive Algorithm

The  backtracking algorithm is designed to **fill schedules based on the order of available time slots**, while strictly respecting all given constraints — including **day and time availability**.

Although Friday and Saturday are provided as valid days in the input, they may not appear in the final schedule because:

* **The algorithm starts filling from the earliest available slots** (typically Monday and Tuesday).
* If all course and lab sessions are successfully assigned within those earlier days, **there’s no need for the algorithm to continue filling the rest of the week**.
* As a result, **Friday and Saturday remain unused**, not because they are invalid, but because the algorithm already found a complete, conflict-free schedule earlier in the week.

This behavior is intentional and efficient:

* It **minimizes scheduling complexity** by avoiding unnecessary spreading across days.
* It **respects the principle of compact timetabling**, which is desirable in small-class scenarios where early-week availability is enough.

In  general  the algorithm doesn’t ignore Friday and Saturday — it simply doesn’t need them, because the required sessions fit neatly into earlier slots like Monday and Tuesday.
